In [27]:
from scipy import sparse
import numpy as np
import requests
import math
import json
import requests

import MDAnalysis
import MDAnalysis.analysis.psa

from sklearn.cluster import DBSCAN ,SpectralClustering
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity

SOLR_URL = "http://localhost:8983/solr"

## Define all function

In [28]:
def segmenting(route):
    '''
    Splits a given route into segments.
    '''
    segments = []
    len_route = len(route)
    for i in range(len_route - 1):
        segments.append([route[i], route[i+1]])
    return segments

def get_slope(points):
    '''
    returns slope for given segment
    '''
    slope = None
    center = None
    length = None
    if float((points[0][0] - points[1][0])) == 0 :
        if float((points[0][1] - points[1][1])) == 0 :
            slope = None
        else:
            slope = 90
    else:
        slope = np.degrees(np.arctan(float((points[0][1] - points[1][1])) / float((points[0][0] - points[1][0]))))
        slope = [slope if slope > 0 else 360 + slope][0]
    return slope

def euc_dist(pt1,pt2):
    '''
    Euclidean distance between two points
    '''
    return math.sqrt((pt2[0]-pt1[0])*(pt2[0]-pt1[0])+(pt2[1]-pt1[1])*(pt2[1]-pt1[1]))


def frechetDist(P,Q):
    ''' 
    Computes the discrete frechet distance between two polygonal lines Algorithm: http://www.kr.tuwien.ac.at/staff/eiter/et-archive/cdtr9464.pdf P and Q are arrays of 2-element arrays (points)
    '''
    def _c(ca,i,j,P,Q):

        if ca[i,j] > -1:
            return ca[i,j]
        elif i == 0 and j == 0:
            ca[i,j] = euc_dist(P[0],Q[0])
        elif i > 0 and j == 0:
            ca[i,j] = max(_c(ca,i-1,0,P,Q),euc_dist(P[i],Q[0]))
        elif i == 0 and j > 0:
            ca[i,j] = max(_c(ca,0,j-1,P,Q),euc_dist(P[0],Q[j]))
        elif i > 0 and j > 0:
            ca[i,j] = max(min(_c(ca,i-1,j,P,Q),_c(ca,i-1,j-1,P,Q),_c(ca,i,j-1,P,Q)),euc_dist(P[i],Q[j]))
        else:
            ca[i,j] = float("inf")
        return ca[i,j]

    ca = np.ones((len(P),len(Q)))
    ca = np.multiply(ca,-1)
    return _c(ca,len(P)-1,len(Q)-1,P,Q)

## Clustering routes based on segments with similar slopes

In [29]:
'''
Define all the slopes that will for the vector of slopes for a route
'''
slope_step = 2
slope_lists = range(0,361,slope_step)
print slope_lists

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 300, 302, 304, 306, 308, 310, 312, 314, 316, 318, 320, 322, 324, 326, 328, 330, 332, 334, 336, 338, 340, 342, 344, 346, 348, 350, 352, 354, 356, 358, 360]


In [30]:
'''
Query solr to get all the routes
'''
phones = {}

url = "http://localhost:8983/solr/raw_data/select?q=*%3A*&wt=json&indent=true&rows=2147483647"
r = requests.get(url)
response = r.json()
docs = response['response']['docs']
for doc in docs:
    phones.setdefault(doc['phone'][0], []).append(doc['location'])

# replace consecutive repeated locations with one
phone_numbers = phones.keys()
for phone_number in phone_numbers:
        refined_location = []
        prev_loc = ""
        for loc in phones[phone_number]:
            if not prev_loc == loc:
                refined_location.append(loc)
            prev_loc=loc
        
        phones[phone_number] = refined_location

# Use routes greater than 4 hops.
lines = {}
for r in phones:
    if len(phones[r]) >= 4:
        lines[r] = phones[r]

In [32]:
'''
Create a slope vector for all the routes, and cluster based on distance between this vector
'''
vector = np.zeros([len(lines), len(slope_lists)])

#dictionary of index to phone_number
index_line = {}

for i, line in enumerate(lines):
    index_line[i] = line
    for seg in segmenting(lines[line]):
        slope = get_slope(seg)
        if slope:
            index = slope_lists.index(int(slope_step * round(float(slope)/slope_step)))
            vector[i][index] = vector[i][index] + 1
            
slope_matrix = np.array(vector)
sparse_matrix = sparse.csr_matrix(slope_matrix)

'''
Find cosine_similarity matrix between all vectors
'''
similarities = cosine_similarity(sparse_matrix)
print('pairwise dense output:\n {}\n'.format(similarities))


'''
Cluster based on cosine_similarity using SpectralClustering.
We can tweak 'num_clusters' to increase/decrease clusters
'''
num_clusters = 100
mat = np.matrix(similarities)
spec_cluster = SpectralClustering(num_clusters).fit_predict(mat)

set_spec_cluster = set(spec_cluster)
print "Clustering done, ", len(spec_cluster) ,"Clusters found"

pairwise dense output:
 [[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]

Clustering done,  717 Clusters found


In [33]:
'''
Group all phone numbers as per cluster assignment.
'''
phone_cluster = []
for each in set_spec_cluster:
    t = []
    if each >= 0 :
        indices = [i for i, x in enumerate(spec_cluster) if x == each]
        for index in indices:
            t.append(index_line[index])
        phone_cluster.append(t)

# print phone_cluster
# print len(phone_cluster[0])

## 2nd level of clustering based on frechet distance

In [34]:
'''
For each route in same cluster from previous step we define distance function between two routes as follows -
1. Find near parallel segments between two routes i.e. segments with similar slope
2. Calculate the Frechet distance between all parrallel segments. 
3. Find top n lowest distances between parallel segments. 
4. Calculate average of top n lowest distances, and return it as distance between two routes.


Cluster routes based on above distance function using DBSCAN 

'''

#reverse index_line dict to access slope_matrix matrix with phone_number
rev_index_line = {v: k for k, v in index_line.iteritems()}

# level 1 means only one common segment is enough to form a cluster
level_to_phone_cluster = {}

for num_common_segments in range(4)[1:]:
    '''
    Loop to find multiple level of clustering
    num_common_segments is a variable which determine how many segments we should consider while computing distance between routes
    '''
    print "Processing level", num_common_segments 
    
    final_phone_cluster = []
    # phone_cluster = [[u'+16187953347', u'+13057411653', u'+19548701222', u'+15594615114']]
    for idx_p, cluster in enumerate(phone_cluster):

        lines_in_clus = []
        for phone_num in cluster:
            lines_in_clus.append(lines[phone_num])

        def get_frechet_dist(i,j):
            i, j = int(i[0]), int(j[0])
            if i == j:
                return 0 

            #Query slope_matrix to get slope vector of both phone_numbers
            phone_i, phone_j = cluster[i], cluster[j]
            idx_i, idx_j = rev_index_line[cluster[i]], rev_index_line[cluster[j]]

            # Find slope which is common in both routes
            parallel_slopes_index = set([])

            for idx,slope_ij in enumerate(zip(slope_matrix[idx_i], slope_matrix[idx_j]) ):
                if slope_ij[0] > 0 and slope_ij[1] > 0 :
                    parallel_slopes_index.add(idx)

            #Print to check parallel slopes are being picked
    #         print A[idx_i]
    #         print A[idx_j]
    #         print parallel_slopes_index

            # Find parallel segments 
            #Key - slope
            #Value - 2 lists, One for each phone_number having segments with same slope
            parallel_segment_pairs = {}

            for idx, line in enumerate( [lines_in_clus[i], lines_in_clus[j]] ) :
                for seg in segmenting(line):
                    slope = get_slope(seg)
                    if slope :
                        index = slope_lists.index(int(slope_step * round(float(slope)/slope_step)))
                        if index in parallel_slopes_index:
                            #initialize with default empty [[],[]] 
                            parallel_segment_pairs[index] = parallel_segment_pairs.get(index, [[],[]])
                            parallel_segment_pairs[index][idx].append(seg)

            # Find distances between all parallel segments


            # initialize distance list. We initialize it with bigger number becasue we will sort the results 
            # and these numbers will be at the end and ignored if length of dist_between_parallel > num_common_segments 
            # if length of dist_between_parallel is < num_common_segments thn we will want to ignore that route from our cluster
            # as it has < num_common_segments parallel segments.

            dist_between_parallel = [1000] * num_common_segments

            for k, parallel_segments_per_phone in parallel_segment_pairs.iteritems():
                parallel_segments_phone1 = parallel_segments_per_phone[0]
                parallel_segments_phone2 = parallel_segments_per_phone[1]

    #             print to check shapes of all segment
    #             print "parallel_segments", parallel_segments_phone1, parallel_segments_phone2

                for seg_phone1 in parallel_segments_phone1:
                    for seg_phone2 in parallel_segments_phone2:
    #                     dist = frechetDist(seg_phone1, seg_phone2)
                        dist = MDAnalysis.analysis.psa.hausdorff(np.array([seg_phone1]), np.array([seg_phone2]))
    #                     print to check shapes of one parallel segment pair
    #                     print seg_phone1, seg_phone2, frechetDist(seg_phone1, seg_phone2)

                        dist_between_parallel.append(dist)


    #         print to check final distance
    #         print "Dist ",dist_between_parallel, min(dist_between_parallel)

    #         sort and pick top n elements 
            dist_between_parallel.sort()
            return np.average(dist_between_parallel[0:num_common_segments])

        X = np.arange(len(lines_in_clus)).reshape(-1, 1)
        dist_matrix = pairwise_distances(X,metric=get_frechet_dist, n_jobs=1)

    #     print "Calculated dist_matrix", dist_matrix.shape

        # EPS can be tweaked
        db = DBSCAN(eps=0.005, min_samples=1, metric='precomputed', n_jobs=-1).fit(dist_matrix)
    #     print 'Number of unique cluster lables', len(np.unique(db.labels_))

        label_idx = [(label,idx) for idx, label in enumerate(db.labels_)]
        label_to_idx = {}
        for k, v in label_idx:
            label_to_idx[k] = label_to_idx.get(k, []) + [cluster[v],]

        # Form list of phone numbers in same cluster.
        for key, value in label_to_idx.iteritems() :
            if key == -1: # Check if it's cluster of noisy values. 
                # Treat them as independednt clusters
    #             print value
                for phone_num in value:
                    '''
                    '''
    #                 final_phone_cluster.append([phone_num])
            else:
                final_phone_cluster.append(value)
        
        # Remove all single clusters
        final_phone_cluster = [c for c in final_phone_cluster if len(c)>1]

        print idx_p+1, "Cluster out of available slope clusters- ", len(phone_cluster), len(final_phone_cluster)
    
    level_to_phone_cluster[num_common_segments] = final_phone_cluster
    

    

Processing level 1
1 Cluster out of available slope clusters-  100 0
2 Cluster out of available slope clusters-  100 44
3 Cluster out of available slope clusters-  100 45
4 Cluster out of available slope clusters-  100 45
5 Cluster out of available slope clusters-  100 45
6 Cluster out of available slope clusters-  100 46
7 Cluster out of available slope clusters-  100 48
8 Cluster out of available slope clusters-  100 48
9 Cluster out of available slope clusters-  100 49
10 Cluster out of available slope clusters-  100 50
11 Cluster out of available slope clusters-  100 50
12 Cluster out of available slope clusters-  100 50
13 Cluster out of available slope clusters-  100 50
14 Cluster out of available slope clusters-  100 51
15 Cluster out of available slope clusters-  100 52
16 Cluster out of available slope clusters-  100 53
17 Cluster out of available slope clusters-  100 53
18 Cluster out of available slope clusters-  100 54
19 Cluster out of available slope clusters-  100 55
20 

Below is one example Color mentioned before routes in text relate to first image. 

Hong Kong -> Luan -> Jiujiang is the common trajectory.
RED :       Wuhan -> Hong Kong -> Luan -> Jiujiang
BLUE :     Fuyang -> Hong Kong -> Luan -> Jiujiang
GREEN :  Luan -> Jiujiang -> Hong Kong -> Luan -> Jiujiang
GREY line shows overlap between these three routes i.e. Hong Kong -> Luan -> Jiujiang
YELLOW lines belong to routes in other clusters

![title](color_route_cluster.png)

Below is raw data, highlighted part showing common trajectory among these routes.

![title](json_route_cluster.png)

## Create json for visualization

In [35]:
url = "{0}/{1}/select?q=*%3A*&wt=json&indent=true&rows=2147483647".format(SOLR_URL, "raw_data")
r = requests.get(url)
response = r.json()
docs = response['response']['docs']

result = []
for level in level_to_phone_cluster:
    for i, cluster in enumerate(level_to_phone_cluster[level]):
        for phone in cluster:
            url = '{0}/{1}/select?q=*%3A*&fq={2}&fl=location&wt=json&rows=2147483647'.format(SOLR_URL, "raw_data", phone)
            r = requests.get(url)
            response = r.json()
            docs = response['response']['docs']
            points = []
            for doc in docs:
                points.append([doc['location'][1], doc['location'][0]])
            result.append({"code": 'a'+str(i), "points": points, "phone":str(phone), "level":str(level)})

with open('level_to_phone_cluster.json', 'w') as outfile:
    json.dump(result, outfile)